In [ ]:
from models import FEAOF
from const import Descriptors
from new_unit import Data, f1_acc, get_config, cross_validate
from tqdm import tqdm
import pandas as pd
from torch_geometric.loader import DataLoader
from typing import List
import torch
import os
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, roc_curve,precision_recall_curve, auc
from math import sqrt
import pickle
from herg_em import fetch_sequence, cal_em
from unit import compute_metrics,predict

In [2]:
Moldel_Type = "FEAOF"
Model = "MPNN"
FingerPrint = "2FP"
Des_IF = "11Des_1441IF_0FC"

file_name = f"./trained_models/{Moldel_Type}/{Model}_{FingerPrint}_{Des_IF}.csv"
with open(file_name,"w") as p:
    p.write("auc_score, pr_auc, AC, f1, SN, PR, SP, CCR, MCC\n")

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

config_path = "./configures/optimized/train_split/MPNN_GRAPH.yml"
model_save_path = "./trained_models/FEAOF/0FC_FEAOF.pkl"

In [4]:
with open(model_save_path, 'rb') as handle:
    model_para = pickle.load(handle)

In [5]:
model_para.keys()

odict_keys(['project_node_feats.0.weight', 'project_node_feats.0.bias', 'gnn_layer.bias', 'gnn_layer.nn.0.weight', 'gnn_layer.nn.0.bias', 'gnn_layer.nn.2.weight', 'gnn_layer.nn.2.bias', 'gnn_layer.lin.weight', 'gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0', 'transformer.lin1.weight', 'transformer.lin1.bias', 'transformer.lin2.weight', 'transformer.lin2.bias', 'transformer.pools.0.S', 'transformer.pools.0.mab.fc_q.weight', 'transformer.pools.0.mab.fc_q.bias', 'transformer.pools.0.mab.layer_k.bias', 'transformer.pools.0.mab.layer_k.lin.weight', 'transformer.pools.0.mab.layer_v.bias', 'transformer.pools.0.mab.layer_v.lin.weight', 'transformer.pools.0.mab.fc_o.weight', 'transformer.pools.0.mab.fc_o.bias', 'transformer.pools.1.mab.fc_q.weight', 'transformer.pools.1.mab.fc_q.bias', 'transformer.pools.1.mab.layer_k.weight', 'transformer.pools.1.mab.layer_k.bias', 'transformer.pools.1.mab.layer_v.weight', 'transformer.pools.1.mab.layer_v.bias', 'transformer.pools.1.

In [6]:
hyperparameters = get_config(config_path)
hyperparameters = {k: v.item() if isinstance(v, np.generic) else v for k, v in hyperparameters.items()}
# model = Cat_GCN(**hyperparameters).model
model = FEAOF(**hyperparameters).model
model.load_state_dict(model_para)

/home/zhaobowen/miniconda3/envs/my_evs/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.GraphMultisetTransformer' is deprecated, use 'nn.aggr.GraphMultisetTransformer' instead
  warnings.warn(out)


<All keys matched successfully>

In [11]:

# uniprot_ID="Q12809"
# uni_id = fetch_sequence(uniprot_ID)
# herg_em = cal_em(uni_id)[0]
herg_em = None

In [18]:
Val_All_Features = "./data/IF_Data/Val_All_Features.pkl"
Val = "./data/processed/Train_Val.csv"
Val = Data(Val)
Val(Descriptors.GRAPH)

./data/processed/Train_Val.csv
data number :  (10543, 4)


In [19]:
y_hat, hiden_features = predict(model,Val.x_test,Val_All_Features,Val.test_Ro5_idx,herg_em,device)
compute_metrics(np.array(Val.y_test),y_hat,file_name)

Binary classification performace metrics:
[[570 100]
 [ 94 353]]
TP, FN, TN, FP
353, 94, 570, 100


(0.8874, 0.8414, 0.8263, 0.7844, 0.7897, 0.7792, 0.8507, 0.8202, 0.6391)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

features_np = hiden_features.detach().cpu().numpy()  # Convert features to numpy array if using PyTorch

labels = np.array(Val.y_test) 
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

tsne = TSNE(n_components=2, random_state=42)
features_tsne = tsne.fit_transform(features_np)

plt.figure(figsize=(10, 8))

scatter = plt.scatter(features_tsne[:, 0], features_tsne[:, 1], c=labels, cmap='coolwarm', marker='o',alpha=0.6)

plt.legend(*scatter.legend_elements(), title="Labels")

plt.title('t-SNE Visualization of Hidden Features of Val')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.show()

In [ ]:
Test_1_All_Features = "./data/IF_Data/Test_1_All_Features.pkl"
Test_1 = "./data/processed/Test_1.csv"
Test_1 = Data(Test_1)
Test_1(Descriptors.GRAPH)

./data/processed/Test_1.csv
data number :  (177, 4)


In [14]:
y_hat, hiden_features = predict(model,Test_1.x_test,Test_1_All_Features,Test_1.test_Ro5_idx,herg_em,device)
compute_metrics(np.array(Test_1.y_test),y_hat,file_name)

Binary classification performace metrics:
[[104  25]
 [ 12  36]]
TP, FN, TN, FP
36, 12, 104, 25


(0.825, 0.5976, 0.791, 0.6606, 0.75, 0.5902, 0.8062, 0.7781, 0.5203)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

features_np = hiden_features.detach().cpu().numpy()  # Convert features to numpy array if using PyTorch

labels = np.array(Test_1.y_test) 
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

tsne = TSNE(n_components=2, random_state=42)
features_tsne = tsne.fit_transform(features_np)

plt.figure(figsize=(10, 8))

scatter = plt.scatter(features_tsne[:, 0], features_tsne[:, 1], c=labels, cmap='coolwarm', marker='o',alpha=0.45)

plt.legend(*scatter.legend_elements(), title="Labels")

plt.title('t-SNE Visualization of Hidden Features of Test_1')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.show()

In [16]:
Test_2_All_Features = "./data/IF_Data/Test_2_All_Features.pkl"
Test_2 = "./data/processed/Test_2.csv"
Test_2 = Data(Test_2)
Test_2(Descriptors.GRAPH)

./data/processed/Test_2.csv
data number :  (379, 4)


In [17]:
y_hat, hiden_features = predict(model,Test_2.x_test,Test_2_All_Features,Test_2.test_Ro5_idx,herg_em,device)
compute_metrics(np.array(Test_2.y_test),y_hat,file_name)

Binary classification performace metrics:
[[224  51]
 [ 24  80]]
TP, FN, TN, FP
80, 24, 224, 51


(0.8395, 0.6314, 0.8021, 0.6809, 0.7692, 0.6107, 0.8145, 0.7919, 0.5477)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


features_np = hiden_features.detach().cpu().numpy()  # Convert features to numpy array if using PyTorch

labels = np.array(Test_2.y_test)  
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

tsne = TSNE(n_components=2, random_state=42)
features_tsne = tsne.fit_transform(features_np)

plt.figure(figsize=(10, 8))
scatter = plt.scatter(features_tsne[:, 0], features_tsne[:, 1], c=labels, cmap='coolwarm', marker='o',alpha=0.6)

plt.legend(*scatter.legend_elements(), title="Labels")

plt.title('t-SNE Visualization of Hidden Features of Test_2')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.show()